<a href="https://colab.research.google.com/github/ThierrryScotto/trabalho_pratico/blob/main/model_training_heat_and_heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
url = 'https://raw.githubusercontent.com/ThierrryScotto/trabalho_pratico/refs/heads/main/datasets/datasets_prontos/dataset_with_geographic_data.csv'

dataset = pd.read_csv(url)

dataset

,date,heat_index,standard_deviation,mean_temperature,mean_humidity,mean_wind_speed,max_temperature,min_temperature,sum_heat_index,percentil_95,...,lag_heat_index_2,lag_heat_index_3,hospitalizations,average_stay,deaths,mortality_rate,deforestation,primary_forest_loss,favela,urbanized_area
0,2000-12,30.079677,4.615565,28.94,51.32,2.00,35.4,22.2,932.47,38.025,...,23.427742,21.801111,4266,6.1,242,5.67,6164,150,1865,1865
1,2001-01,31.907742,4.864888,29.69,59.10,2.19,35.4,21.4,989.14,38.780,...,26.305667,23.427742,4032,5.9,218,5.41,6390,201,1903,14972
2,2001-02,32.959643,3.889432,30.08,60.14,2.33,34.2,23.5,922.87,38.625,...,30.079677,26.305667,3888,5.8,217,5.58,6390,201,1903,14972
3,2001-03,32.071290,3.585808,29.82,56.61,2.21,34.4,25.1,994.21,37.065,...,31.907742,30.079677,3658,5.8,176,4.81,6390,201,1903,14972
4,2001-04,25.963667,3.307106,25.37,64.77,1.68,30.8,19.3,778.91,30.751,...,32.959643,31.907742,4022,5.8,256,6.36,6390,201,1903,14972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2023-08,22.680968,5.464467,22.86,52.13,1.62,32.3,15.2,703.11,31.335,...,21.014000,23.174194,3768,6.9,344,9.13,8843,12,2133,16874
273,2023-09,23.058000,5.559752,22.70,69.97,1.69,34.9,15.6,691.74,33.460,...,19.769677,21.014000,3606,6.9,329,9.12,8843,12,2133,16874
274,2023-10,23.510645,3.787990,23.30,63.58,1.84,31.6,16.9,728.83,29.465,...,22.680968,19.769677,3697,7.5,341,9.22,8843,12,2133,16874
275,2023-11,26.595000,3.669000,25.96,61.17,1.65,33.5,19.9,797.85,31.954,...,23.058000,22.680968,3543,7.3,342,9.65,8843,12,2133,16874


## Treino e test

In [ ]:
features = dataset.columns.tolist()
features.remove('date')

In [ ]:
dataset.drop(columns=['date'], inplace=True)

X = dataset[features]
y = dataset['hospitalizations'].values

X

,heat_index,standard_deviation,mean_temperature,mean_humidity,mean_wind_speed,max_temperature,min_temperature,sum_heat_index,percentil_95,percentil_99,...,lag_heat_index_2,lag_heat_index_3,hospitalizations,average_stay,deaths,mortality_rate,deforestation,primary_forest_loss,favela,urbanized_area
0,30.079677,4.615565,28.94,51.32,2.00,35.4,22.2,932.47,38.025,38.5520,...,23.427742,21.801111,4266,6.1,242,5.67,6164,150,1865,1865
1,31.907742,4.864888,29.69,59.10,2.19,35.4,21.4,989.14,38.780,39.5900,...,26.305667,23.427742,4032,5.9,218,5.41,6390,201,1903,14972
2,32.959643,3.889432,30.08,60.14,2.33,34.2,23.5,922.87,38.625,38.8030,...,30.079677,26.305667,3888,5.8,217,5.58,6390,201,1903,14972
3,32.071290,3.585808,29.82,56.61,2.21,34.4,25.1,994.21,37.065,37.7400,...,31.907742,30.079677,3658,5.8,176,4.81,6390,201,1903,14972
4,25.963667,3.307106,25.37,64.77,1.68,30.8,19.3,778.91,30.751,32.1117,...,32.959643,31.907742,4022,5.8,256,6.36,6390,201,1903,14972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,22.680968,5.464467,22.86,52.13,1.62,32.3,15.2,703.11,31.335,33.0020,...,21.014000,23.174194,3768,6.9,344,9.13,8843,12,2133,16874
273,23.058000,5.559752,22.70,69.97,1.69,34.9,15.6,691.74,33.460,36.6514,...,19.769677,21.014000,3606,6.9,329,9.12,8843,12,2133,16874
274,23.510645,3.787990,23.30,63.58,1.84,31.6,16.9,728.83,29.465,32.1660,...,22.680968,19.769677,3697,7.5,341,9.22,8843,12,2133,16874
275,26.595000,3.669000,25.96,61.17,1.65,33.5,19.9,797.85,31.954,34.2029,...,23.058000,22.680968,3543,7.3,342,9.65,8843,12,2133,16874


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(193, 21)
(84, 21)
(193,)
(84,)


## Normalização

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))  # Ajusta e transforma no treino
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))  # Transforma no teste

# df_scaled = scaler.transform(X_train)

In [ ]:
# Exibindo os dados escalados
print("X_train_scaled:\n", X_train_scaled)
print("y_train_scaled:\n", y_train_scaled)
print("X_test_scaled:\n", X_test_scaled)
print("y_test_scaled:\n", y_test_scaled)

X_train_scaled:
 [[0.67165559 0.07159067 0.76850207 ... 0.21985816 0.46268657 0.90558998]
 [0.38778021 0.43910492 0.44227353 ... 0.02836879 0.87686567 0.98540875]
 [0.65770869 0.21094426 0.7785672  ... 0.17730496 0.74626866 0.96708641]
 ...
 [0.43825424 0.52864214 0.51805802 ... 0.15780142 0.67537313 0.94170165]
 [0.285456   0.50931079 0.32978094 ... 0.         1.         1.        ]
 [0.10735795 0.35332233 0.13262285 ... 0.15780142 0.67537313 0.94170165]]
y_train_scaled:
 [[0.56920684]
 [0.48911353]
 [0.49572317]
 [0.52566096]
 [0.35147745]
 [0.43001555]
 [0.47900467]
 [0.70023328]
 [0.22978227]
 [0.6722395 ]
 [0.25699844]
 [0.57387247]
 [0.43234837]
 [0.40979782]
 [0.35692068]
 [0.437014  ]
 [0.53888025]
 [0.48716952]
 [0.33748056]
 [0.69790047]
 [0.50894246]
 [0.65785381]
 [0.50816485]
 [0.49572317]
 [0.3755832 ]
 [0.27566096]
 [0.32892691]
 [0.47978227]
 [0.59486781]
 [0.4529549 ]
 [0.47433904]
 [0.45839813]
 [0.4537325 ]
 [0.52527216]
 [0.7503888 ]
 [0.54976672]
 [0.54432348]
 [0.

## Treinamento

In [ ]:
model = XGBRegressor(
    n_estimators=25,  # Número de árvores
    learning_rate=0.01,  # Taxa de aprendizado
    max_depth=3,  # Profundidade máxima da árvore
    random_state=42
)

# Treinar o modelo com os dados escalados
model.fit(X_train_scaled, y_train_scaled)

# Fazer previsões no conjunto de teste
y_pred_scaled = model.predict(X_test_scaled)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test_scaled.reshape(-1), y_pred_scaled)

In [ ]:
y_test_scaled = y_test_scaled.reshape(-1)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cálculo das métricas
mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
mse = mean_squared_error(y_test_scaled, y_pred_scaled)
rmse = mean_squared_error(y_test_scaled, y_pred_scaled, squared=False)  # squared=False retorna a raiz
mape = np.mean(np.abs((y_test_scaled - y_pred_scaled) / y_test_scaled)) * 100
r2 = r2_score(y_test_scaled, y_pred_scaled)

# Exibir os resultados
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R-Squared (R²): {r2:.2f}")


Mean Absolute Error (MAE): 0.10
Mean Squared Error (MSE): 0.02
Root Mean Squared Error (RMSE): 0.12
Mean Absolute Percentage Error (MAPE): 21.87%
R-Squared (R²): 0.37


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo os hiperparâmetros para busca
param_grid = {
    'n_estimators': [25, 50, 100, 200],  # Número de árvores
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Taxa de aprendizado
    'max_depth': [3, 5, 7],  # Profundidade máxima das árvores
    'subsample': [0.6, 0.8, 1.0],  # Taxa de amostragem
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fração de colunas por árvore
    'gamma': [0, 0.1, 0.2]  # Reduz o crescimento da árvore
}

# Instanciar o modelo
model = XGBRegressor(random_state=42)

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Métrica de avaliação
    cv=3,  # Número de divisões na validação cruzada
    verbose=1,  # Exibe o progresso
    n_jobs=-1  # Usa todos os núcleos disponíveis
)

# Ajustar o modelo com os dados de treino
grid_search.fit(X_train_scaled, y_train_scaled)

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

# Melhor modelo ajustado
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 1296 candidates, totalling 3888 fits
Melhores hiperparâmetros encontrados:
{'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.6}


In [ ]:
model = XGBRegressor(
    n_estimators=200,  # Número de árvores
    colsample_bytree=1.0,
    gamma=0,
    learning_rate=0.05,  # Taxa de aprendizado
    max_depth=7,  # Profundidade máxima da árvore
    random_state=42,
    subsample= 0.6
)

# Treinar o modelo com os dados escalados
model.fit(X_train_scaled, y_train_scaled)

# Fazer previsões no conjunto de teste
y_pred_scaled = model.predict(X_test_scaled)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test_scaled.reshape(-1), y_pred_scaled)

In [ ]:
# Cálculo das métricas
mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
mse = mean_squared_error(y_test_scaled, y_pred_scaled)
rmse = mean_squared_error(y_test_scaled, y_pred_scaled, squared=False)  # squared=False retorna a raiz
mape = np.mean(np.abs((y_test_scaled - y_pred_scaled) / y_test_scaled)) * 100
r2 = r2_score(y_test_scaled, y_pred_scaled)

# Exibir os resultados
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R-Squared (R²): {r2:.2f}")


Mean Absolute Error (MAE): 0.00
Mean Squared Error (MSE): 0.00
Root Mean Squared Error (RMSE): 0.01
Mean Absolute Percentage Error (MAPE): 1.16%
R-Squared (R²): 1.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
